# TP Analyse musculo squelettique - cinématique
**Charles Pontonnier - Nicolas Bideau**

L'objectif de ce TP sur 3 séances est de réaliser l'analyse cinématique, dynamique et musculaire d'une capture de mouvement de marche. Ce premier script python se concentre sur la réalisation de la calibration géométrique du modèle ainsi que la cinématique inverse permettant de récupérer, au cours du temps, les trajectoires articulaires.


## Setup du Notebook

In [3]:
import ezc3d 
import numpy as np
import matplotlib.pyplot as plt
from scipy import signal
import os


## Exploration des données d'entrée

Avant de lancer une analyse musculosquelettique, nous allons explorer les données à disposition.

Dans le dossier DATA, charger le fichier <font color='red'>"Marche0007.c3d"</font> sous MOKKA. Observer les données disponibles.

Etant donné que l'objet de l'étude est une marche à vitesse de confort, il est important dès maintenant d'analyser les données disponibles en termes de cycle de marche. Pour rappel, on se fiera à la définition suivante pour définir un cycle de marche (au pied de départ près).

Srinivasan, Sujatha, I. A. Raptis, and Eric R. Westervelt. "Low-dimensional sagittal plane model of normal human walking." Journal of biomechanical engineering 130.5 (2008).

![alt text](gait_angles.jpg "données de référence flexion membre inférieur")


<font color='green'>**Question 1: Compter le nombre de cycles complets disponibles dans cet exemple, et identifier les frames de début et fin de cycle. Noter également quel pied se pose sur quelle plateforme et pour quel cycle les données de plateforme sont disponibles.**</font>




## Préparation des données d'entrée

Tout d'abord, il est nécessaire d'exporter les données de capture afin de permettre leur lecture par opensim, qui ne lit pas les C3D.

<font color='green'>**Question 2: Dans MOKKA, utiliser la fonction export pour exporter le C3D dans le format .trc, qui est le format de données lu par OpenSim pour les données de marqueurs.**</font>

Ensuite, étant donné qu'Opensim ne sait traiter des données de capture uniquement orientées y-up, il est nécessaire de modifier les données d'entrée pour les adapter au solveur.

![alt text](orientation_frames.png "orientation des données dans opensim")

<font color='green'>**Question 3: Dans Opensim, ouvrir le fichier .trc (<font color='red'>File>Preview Experimental Data...</font>) et utiliser la fonction "Transform" pour orienter les données "y-up" avec x comme direction principale de marche, comme représenté ci-dessus. Utiliser le bouton "save as" pour sauver les données au format .trc.**</font>

Enfin, il est nécessaire pour la dynamique inverse (traitée plus tard) de générer un fichier contenant les efforts de réaction au sol (Ground Reaction Forces, GRF en anglais). Ce traitement a déjà été réalisé pour l'essai que vous traitez dans ce TP et est disponible dans DATA, sous le nom <font color='red'>"GRF_marche0007.mot"</font>.

<font color='green'>**Question 4: Répéter les questions 2 et 3 pour le fichier <font color='red'>"statique.c3d"</font>.**</font>

## Exploration du modèle opensim

A présent, vous pouvez fermer les données expérimentales et vous concentrer sur le modèle musculo-squelettique que nous allons exploiter par la suite. Pour cela, ouvrez le modèle disponible à la racine du dossier <font color='red'>"M2S_model_complet.osim"</font>.

Vous devriez avoir à présent sous vos yeux le modèle suivant:

![alt text](model.png "modèle M2S")

<font color='green'>**Question 5: A l'aide de l'onglet <font color='red'>coordinates</font>, explorer les degrés de liberté du modèle. Le modèle présente-t-il des muscles sur tous ses segments ? **</font>.

## Scaling du modèle

Le modèle étant chargé, nous allons à présent réaliser sa calibration géométrique (scaling). Pour celà, aller dans <font color='red'>Tools>Scale model...</font>. 

<font color='green'>**Question 6: A l'aide des instructions données en direct, mettre en oeuvre une mise à l'échelle cohérente. Pensez-à bien sauver vos "settings" au format XML. Lancer le scaling. Qu'en déduisez-vous ?**</font>.

## Cinématique inverse

A présent que le modèle a été mis à l'échelle, nous allons réaliser l'étape de cinématique inverse.

<font color='green'>**Question 7: A l'aide des instructions données en direct, mettre en oeuvre la méthode de cinématique inverse. Pensez-à bien sauver vos "settings" au format XML. Lancer le calcul. Que déduisez-vous des valeurs d'erreur obtenues ?**</font>

## Post processing

A présent, nous avons calculé l'ensemble des trajectoires articulaires au cours du temps. Ces étapes ayant été finalisées et les résultats ayant été sauvegardés, il est tout d'abord possible de les visualiser dans Opensim

<font color='green'>**Question 8: A l'aide de l'outil <font color='red'>Tools>Plot...</font>, tracer les angles de flexion de hanche, genou et cheville au cours du temps. Les allures obtenues vous paraissent-elles conformes à celles présentées en début de TP ?**</font>

Nous allons à présent exporter les données de cinématique inverse dans ce notebook pour pouvoir normaliser et superposer les cycles de marche.

In [2]:
# Fonction de récupération des données d'OpenSim
def readMotionFile(filename):
    """ Reads OpenSim .sto files.
    Parameters
    ----------
    filename: absolute path to the .sto file
    Returns
    -------
    header: the header of the .sto
    labels: the labels of the columns
    data: an array of the data
    """

    if not os.path.exists(filename):
        print('file do not exists')

    file_id = open(filename, 'r')

    # read header
    next_line = file_id.readline()
    header = [next_line]
    nc = 0
    nr = 0
    while not 'endheader' in next_line:
        if 'datacolumns' in next_line:
            nc = int(next_line[next_line.index(' ') + 1:len(next_line)])
        elif 'datarows' in next_line:
            nr = int(next_line[next_line.index(' ') + 1:len(next_line)])
    
        elif 'nColumns' in next_line:
            nc = int(next_line[next_line.index('=') + 1:len(next_line)])
        elif 'nRows' in next_line:
            nr = int(next_line[next_line.index('=') + 1:len(next_line)])

        next_line = file_id.readline()
        header.append(next_line)

    # process column labels
    next_line = file_id.readline()
    if next_line.isspace() == True:
        next_line = file_id.readline()

    labels = next_line.split()

    # get data
    data = []
    for i in range(1, nr + 1):
        d = [float(x) for x in file_id.readline().split()]
        data.append(d)

    file_id.close()

    return header, labels, data


#récupération des données de cinématique
#[h,l,d]=readMotionFile('../Trial010_IK.mot')
#impression des labels
#print(l)

#récupération des données "épaule"
#flex_shoulder_l=np.zeros(len(d))
#flex_elbow_l=np.zeros(len(d))
#add_shoulder_l=np.zeros(len(d))
#time=np.zeros(len(d))
#for i in range(len(d)):
 #   flex_shoulder_l[i]=d[i][33]
 #   flex_elbow_l[i]=d[i][36]
 #   add_shoulder_l[i]=d[i][34]
 #   time[i]=d[i][0]

<font color='green'>**Question 9: A l'aide du code ci-dessus, récupérer les résultats de la cinématique inverse pour la flexion de hanche, de genou et de cheville.**</font>


<font color='green'>**Question 10: A l'aide des informations recueillies au tout début du TP, découper les résultats en cycles, normaliser le temps de cycle et tracer sur un même graphe les angles articulaires obtenus pour chaque cycle. Commentez votre **</font>